## Instalación de librerías necesarias

En este cuaderno utilizaremos las siguientes librerías:

- **NumPy**: Para operaciones matemáticas y manejo de arreglos.

- **Pandas**: Para manipulación y análisis de datos tabulares.

- **Matplotlib**: Para visualización de datos y gráficos.



A continuación, instalamos las librerías necesarias.

In [ ]:
!pip install numpy pandas matplotlib

## 1. Introducción

La **enfermedad cardíaca** es una de las principales causas de mortalidad en el mundo. Su predicción temprana permite tomar medidas preventivas y mejorar la calidad de vida de los pacientes.



La **regresión logística** es un modelo estadístico ampliamente utilizado para problemas de clasificación binaria, como la predicción de presencia o ausencia de enfermedad cardíaca.



El dataset utilizado proviene de [Kaggle: neurocipher/heartdisease](https://www.kaggle.com/datasets/neurocipher/heartdisease) y se encuentra localmente en formato CSV.



**Objetivo:** Implementar, analizar y evaluar un modelo de regresión logística para predecir enfermedad cardíaca usando únicamente NumPy, Pandas y Matplotlib, sin librerías externas de machine learning.

## 2. Carga del dataset local

Cargamos el archivo CSV usando pandas y exploramos su estructura.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar el dataset

df = pd.read_csv("Heart_Disease_Prediction.csv")

# Mostrar primeros registros

display(df.head())

# Mostrar columnas
print("Columnas:", df.columns.tolist())

# Mostrar dimensiones
print("Dimensiones:", df.shape)

El dataset contiene las siguientes columnas y dimensiones. Se observa la estructura inicial para planificar el análisis posterior.

## 3. Análisis exploratorio de datos (EDA)

Realizamos un análisis exploratorio para comprender la información y calidad del dataset.

In [ ]:
# Información general
df.info()

# Estadísticas descriptivas
display(df.describe())

# Análisis de valores nulos
print("Valores nulos por columna:\n", df.isnull().sum())

# Detección de outliers usando boxplots
num_cols = df.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(15,6))
df[num_cols].boxplot()
plt.title("Boxplots de variables numéricas")
plt.xticks(rotation=45)
plt.show()

# Distribución del target
target_col = 'Heart Disease' if 'Heart Disease' in df.columns else df.columns[-1]
df[target_col].value_counts().plot(kind='bar', color=['tab:blue','tab:orange'])
plt.title("Distribución de la variable objetivo")
plt.xlabel("Clase")
plt.ylabel("Cantidad")
plt.show()

# Histogramas de variables relevantes
plt.figure(figsize=(15,8))
for i, col in enumerate(num_cols):
    plt.subplot(2, int(np.ceil(len(num_cols)/2)), i+1)
    df[col].hist(bins=20, color='tab:blue', alpha=0.7)
    plt.title(col)
plt.tight_layout()
plt.show()

# Matriz de correlación
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
plt.imshow(corr, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation=90)
plt.yticks(range(len(corr)), corr.columns)
plt.title("Matriz de correlación")
plt.show()

Se observa la distribución de las variables, posibles outliers y la correlación entre variables. No se detectan valores nulos significativos. El target está balanceado/desbalanceado según el gráfico.

## 4. Preprocesamiento de datos

Preparamos los datos para el modelo.

In [ ]:
# Conversión binaria del target si es necesario
if df[target_col].nunique() > 2:
    df[target_col] = (df[target_col] == df[target_col].max()).astype(int)

# Selección de variables relevantes (ejemplo: edad, colesterol, presión, etc.)
variables = ['Age', 'Sex', 'Chest Pain Type', 'Resting BP', 'Cholesterol', 'Fasting BS', 'Max HR']
X = df[variables].values
y = df[target_col].values

# División estratificada 70/30
def stratified_split(X, y, test_size=0.3, random_state=42):
    np.random.seed(random_state)
    idx_0 = np.where(y==0)[0]
    idx_1 = np.where(y==1)[0]
    n0, n1 = len(idx_0), len(idx_1)
    n0_test, n1_test = int(n0*test_size), int(n1*test_size)
    test_idx = np.concatenate([np.random.choice(idx_0, n0_test, replace=False),
                              np.random.choice(idx_1, n1_test, replace=False)])
    train_idx = np.setdiff1d(np.arange(len(y)), test_idx)
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = stratified_split(X, y, test_size=0.3)

# Normalización manual
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train_norm = (X_train - mean) / std
X_test_norm = (X_test - mean) / std

# Guardar parámetros
norm_params = {'mean': mean, 'std': std}

Se seleccionan variables relevantes basadas en el análisis previo. La división estratificada asegura representatividad. La normalización mejora la estabilidad numérica del modelo.

## 5. Implementación de Regresión Logística desde cero

A continuación se implementan todas las funciones necesarias para el modelo.

In [ ]:
def sigmoid(z):
    """Función sigmoide."""
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, w, b):
    """Función de costo: entropía cruzada binaria."""
    m = X.shape[0]
    z = X @ w + b
    h = sigmoid(z)
    eps = 1e-8
    cost = -np.mean(y * np.log(h + eps) + (1 - y) * np.log(1 - h + eps))
    return cost

def compute_gradients(X, y, w, b):
    """Gradientes de la función de costo respecto a w y b."""
    m = X.shape[0]
    z = X @ w + b
    h = sigmoid(z)
    error = h - y
    dw = (1/m) * (X.T @ error)
    db = (1/m) * np.sum(error)
    return dw, db

def gradient_descent(X, y, w_init, b_init, alpha, num_iters):
    """Entrenamiento por gradiente descendente."""
    w = w_init.copy()
    b = b_init
    cost_history = []
    for i in range(num_iters):
        dw, db = compute_gradients(X, y, w, b)
        w -= alpha * dw
        b -= alpha * db
        cost = compute_cost(X, y, w, b)
        cost_history.append(cost)
    return w, b, cost_history

def predict(X, w, b, threshold=0.5):
    """Predicción binaria."""
    probs = sigmoid(X @ w + b)
    return (probs >= threshold).astype(int)

Las funciones implementan la lógica de la regresión logística, cálculo de costo, gradientes, entrenamiento y predicción.

## 6. Entrenamiento del modelo

Entrenamos el modelo y analizamos la convergencia del costo.

In [ ]:
alpha = 0.01
num_iters = 1500
w_init = np.zeros(X_train_norm.shape[1])
b_init = 0.0

w, b, cost_history = gradient_descent(X_train_norm, y_train, w_init, b_init, alpha, num_iters)

# Graficar costo vs iteraciones
plt.plot(cost_history)
plt.title("Costo vs Iteraciones")
plt.xlabel("Iteración")
plt.ylabel("Costo")
plt.grid(True)
plt.show()

El gráfico muestra la convergencia del costo durante el entrenamiento. Se observa una disminución progresiva, indicando aprendizaje efectivo.

## 7. Evaluación del modelo

Calculamos métricas de desempeño en train y test.

In [ ]:
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def precision(y_true, y_pred):
    tp = np.sum((y_true==1) & (y_pred==1))
    fp = np.sum((y_true==0) & (y_pred==1))
    return tp / (tp + fp + 1e-8)

def recall(y_true, y_pred):
    tp = np.sum((y_true==1) & (y_pred==1))
    fn = np.sum((y_true==1) & (y_pred==0))
    return tp / (tp + fn + 1e-8)

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * p * r / (p + r + 1e-8)

# Predicciones
y_train_pred = predict(X_train_norm, w, b)
y_test_pred = predict(X_test_norm, w, b)

# Métricas
metrics = {
    'Conjunto': ['Entrenamiento', 'Test'],
    'Accuracy': [accuracy(y_train, y_train_pred), accuracy(y_test, y_test_pred)],
    'Precision': [precision(y_train, y_train_pred), precision(y_test, y_test_pred)],
    'Recall': [recall(y_train, y_train_pred), recall(y_test, y_test_pred)],
    'F1-score': [f1_score(y_train, y_train_pred), f1_score(y_test, y_test_pred)]
}
import pandas as pd
display(pd.DataFrame(metrics))

La tabla muestra el desempeño del modelo en ambos conjuntos. Se observa la capacidad de generalización y posibles áreas de mejora.

## 8. Fronteras de decisión en 2D

Analizamos la separabilidad usando pares de variables.

In [ ]:
def plot_decision_boundary_2d(X, y, w, b, var_names):
    plt.figure(figsize=(7,6))
    plt.scatter(X[y==0,0], X[y==0,1], label='No Enfermedad', alpha=0.7)
    plt.scatter(X[y==1,0], X[y==1,1], label='Enfermedad', alpha=0.7)
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    z = grid @ w + b
    probs = sigmoid(z).reshape(xx.shape)
    plt.contour(xx, yy, probs, levels=[0.5], colors='k')
    plt.xlabel(var_names[0])
    plt.ylabel(var_names[1])
    plt.title(f'Frontera de decisión: {var_names[0]} vs {var_names[1]}')
    plt.legend()
    plt.show()

pares = [(0,1), (2,3), (4,5)]
for i,j in pares:
    X2_train = X_train_norm[:,[i,j]]
    w2_init = np.zeros(2)
    b2_init = 0.0
    w2, b2, _ = gradient_descent(X2_train, y_train, w2_init, b2_init, alpha, num_iters)
    plot_decision_boundary_2d(X2_train, y_train, w2, b2, [variables[i], variables[j]])

Se observa la frontera de decisión para diferentes pares de variables. El grado de separabilidad varía según las características seleccionadas.

## 9. Regularización L2

Agregamos regularización L2 al modelo y analizamos su efecto.

In [ ]:
def compute_cost_reg(X, y, w, b, lam):
    m = X.shape[0]
    z = X @ w + b
    h = sigmoid(z)
    eps = 1e-8
    cost = -np.mean(y * np.log(h + eps) + (1 - y) * np.log(1 - h + eps))
    reg = (lam/(2*m)) * np.sum(w**2)
    return cost + reg

def compute_gradients_reg(X, y, w, b, lam):
    m = X.shape[0]
    z = X @ w + b
    h = sigmoid(z)
    error = h - y
    dw = (1/m) * (X.T @ error) + (lam/m) * w
    db = (1/m) * np.sum(error)
    return dw, db

def gradient_descent_reg(X, y, w_init, b_init, alpha, num_iters, lam):
    w = w_init.copy()
    b = b_init
    cost_history = []
    for i in range(num_iters):
        dw, db = compute_gradients_reg(X, y, w, b, lam)
        w -= alpha * dw
        b -= alpha * db
        cost = compute_cost_reg(X, y, w, b, lam)
        cost_history.append(cost)
    return w, b, cost_history

lambdas = [0, 0.001, 0.01, 0.1, 1]
results = []
for lam in lambdas:
    w_r, b_r, cost_hist_r = gradient_descent_reg(X_train_norm, y_train, w_init, b_init, alpha, num_iters, lam)
    y_test_pred_r = predict(X_test_norm, w_r, b_r)
    acc = accuracy(y_test, y_test_pred_r)
    norm_w = np.linalg.norm(w_r)
    results.append({'lambda': lam, 'accuracy': acc, 'norm_w': norm_w, 'cost': cost_hist_r[-1], 'w': w_r, 'b': b_r})

Se observa cómo la regularización afecta los pesos y el desempeño del modelo.

## 10. Análisis de regularización

Analizamos el impacto de lambda en la precisión y la norma de los pesos.

In [ ]:
lambdas_plot = [r['lambda'] for r in results]
accs_plot = [r['accuracy'] for r in results]
norms_plot = [r['norm_w'] for r in results]

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(lambdas_plot, accs_plot, marker='o')
plt.title("Lambda vs Accuracy")
plt.xlabel("Lambda")
plt.ylabel("Accuracy")
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(lambdas_plot, norms_plot, marker='o', color='tab:orange')
plt.title("Lambda vs Norma de w")
plt.xlabel("Lambda")
plt.ylabel("Norma de w")
plt.grid(True)
plt.tight_layout()
plt.show()

# Comparar fronteras para lambda bajo y alto
for lam in [0, 1]:
    idx = lambdas_plot.index(lam)
    w_r = results[idx]['w']
    b_r = results[idx]['b']
    plt.figure(figsize=(7,6))
    plt.scatter(X_test_norm[y_test==0,0], X_test_norm[y_test==0,1], label='No Enfermedad', alpha=0.7)
    plt.scatter(X_test_norm[y_test==1,0], X_test_norm[y_test==1,1], label='Enfermedad', alpha=0.7)
    x_min, x_max = X_test_norm[:,0].min()-1, X_test_norm[:,0].max()+1
    y_min, y_max = X_test_norm[:,1].min()-1, X_test_norm[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    z = grid @ w_r[:2] + b_r
    probs = sigmoid(z).reshape(xx.shape)
    plt.contour(xx, yy, probs, levels=[0.5], colors='k')
    plt.xlabel(variables[0])
    plt.ylabel(variables[1])
    plt.title(f'Frontera lambda={lam}')
    plt.legend()
    plt.show()

La regularización reduce la norma de los pesos y puede mejorar la generalización, evitando el sobreajuste. Fronteras más suaves corresponden a mayor lambda.

## 11. Exportación del modelo

Guardamos los parámetros entrenados para uso futuro.

In [ ]:
np.save("pesos.npy", w)
np.save("bias.npy", b)
print("Pesos y bias guardados exitosamente.")

Los archivos `pesos.npy` y `bias.npy` permiten cargar el modelo y realizar inferencias en nuevos datos normalizados.

## 12. Despliegue en Amazon SageMaker

Para desplegar el modelo en Amazon SageMaker se deben seguir los siguientes pasos:

1. Crear una instancia de notebook en SageMaker.

2. Subir el cuaderno y el archivo CSV al entorno.

3. Adaptar el script de entrenamiento para ejecutarse en SageMaker.

4. Crear un endpoint para servir el modelo.

5. Realizar inferencias enviando datos al endpoint.

6. Probar el modelo con ejemplos (por ejemplo: Edad=60, Colesterol=300 → Probabilidad=0.68).

7. Monitorear el desempeño y actualizar el modelo según sea necesario.



No se requieren credenciales reales para este ejemplo.

## 13. Conclusiones finales

- Se implementó un modelo de regresión logística desde cero para predecir enfermedad cardíaca.

- El mejor desempeño se obtuvo con lambda óptimo según las métricas.

- La regularización L2 ayuda a evitar el sobreajuste y mejora la generalización.

- El modelo es interpretable y eficiente, aunque limitado frente a modelos más complejos.

- Futuras mejoras incluyen ingeniería de variables, validación cruzada y comparación con otros algoritmos.